42 プライベートな属性よりパブリックな属性が好ましい

In [4]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field

In [5]:
foo = MyObject()
assert foo.public_field==5

In [6]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [7]:
assert foo.get_private_field()==10

In [8]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

In [10]:
assert baz._MyParentObject__private_field==71

In [12]:
baz.__dict__

{'_MyParentObject__private_field': 71}

In [13]:
#プライベート属性を使うことを考えるのはサブクラスとの名前の衝突を心配するときのみ
class ApiClass:
    def __init__(self):
        self._value= 5

    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

a= Child()
print(f'{a.get()} and {a._value} should be different')
#衝突してしまう

hello and hello should be different


In [15]:
#衝突を避けるためのプライベート属性
class ApiClass:
    def __init__(self):
        self.__value=5

    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'
a = Child()
print(f'{a.get()} and {a._value} are different')

5 and hello are different


カスタムコンテナ型はcollections.abcを継承する

In [16]:
#要素の頻度を数える追加メソッドを持ったカスタムリスト
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)

    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0)+1
        return counts

In [18]:
foo = FrequencyList(['a','b','a','c','b','a','d'])
print('Length',len(foo))
foo.pop()
print('after pop', repr(foo))
print('Frequency:',foo.frequency())

Length 7
after pop ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


In [49]:
#シーケンスのようにふるまわせるには__getitem__の実装を提供する
class BinaryNode:
    def __init__(self, value, left=None, right= None):
        self.value = value
        self.left = left
        self.right = right

class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i==index:
                return item.value
        raise IndexError(f'Index {index} is out of range')


In [50]:
#__len__を実装すると長さが返せる
#それでもcountやindexが欠けている
#collections.abcがコンテナ型の典型的なメソッドをすべて提供する抽象基底クラスを定義する

from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType() 

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [51]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

In [52]:
tree = SequenceNode(
    10,
    left = SequenceNode(
        5,
        left = SequenceNode(2),
        right = SequenceNode(6, right = SequenceNode(7))),
    right= SequenceNode(
        15,
        left = SequenceNode(11)
    )
)

In [53]:
tree[1]

5

In [54]:
print('Tree length:', len(tree))

Tree length: 7


In [55]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left = BetterNode(
        5,
        left=BetterNode(2),
        right = BetterNode(6,
                            right=BetterNode(7))),
    right= BetterNode(
        15,
        left = BetterNode(11)
    )
)


In [56]:
tree[1]

5

In [57]:
tree.index(7)

3

In [58]:
tree.count(10)

1